## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [3]:
# answer goes here
data = 'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv'
df = pd.read_csv(data)

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [64]:
# answer goes here
df20st = df[df.GS > 19].reset_index()
df10min = df20st[df20st.MP > 9.9].reset_index()
df10min

,level_0,index,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
1,1,4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
2,2,7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
3,3,10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
4,4,12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,800,2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
800,801,2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
801,802,2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
802,803,2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [65]:
# answer goes here
X = df10min[['PTS','TRB','AST','STL','BLK','TOV','FG%']]

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [67]:
# answer goes here
scale = StandardScaler()
scaled = pd.DataFrame(scale.fit_transform(X), columns=X.columns)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [68]:
# answer goes here
kmeans3 = KMeans(n_clusters=3)
kmeans3.fit_predict(scaled)
kclust3 = scale.inverse_transform(kmeans3.cluster_centers_)
kclust3 = pd.DataFrame(kclust3, columns=scaled.columns)
kclust3.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%
0,19.580337,5.351124,5.922472,1.364045,0.503933,2.800562,0.451511
1,10.859861,3.990951,2.144780,0.830626,0.361021,1.270302,0.438450
2,12.685128,8.134872,1.776923,0.715897,1.182564,1.565128,0.538779


In [69]:
kmeans7 = KMeans(n_clusters=7)
kmeans7.fit_predict(scaled)
kclust7 = scale.inverse_transform(kmeans7.cluster_centers_)
kclust7 = pd.DataFrame(kclust7, columns=scaled.columns)
kclust7.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%
0,15.691270,6.415873,2.371429,0.918254,0.565079,1.666667,0.464444
1,9.413953,6.627907,1.361240,0.590698,0.963566,1.203876,0.539186
2,17.551667,10.608333,2.176667,0.931667,1.728333,2.101667,0.546900
3,24.560000,8.008571,7.617143,1.580000,0.800000,3.862857,0.478943
4,12.014839,3.446452,3.183871,1.053548,0.276129,1.582581,0.429348
5,8.239326,3.368539,1.451124,0.637640,0.337640,0.920787,0.431022
6,18.234711,4.503306,5.799174,1.326446,0.378512,2.606612,0.441983


In [70]:
X_c3 = X.copy(deep=True)

In [71]:
gmm3 = GaussianMixture(n_components=3, covariance_type='full')
gmm3.fit(scaled)
X_c3['gmm3_cluster'] = gmm3.predict(scaled)
gmmprob3 = pd.DataFrame(gmm3.predict_proba(scaled))
X_c3 = pd.concat([X_c3, gmmprob3], axis=1)

gmmcenter3 = pd.DataFrame(gmm3.means_, columns=scaled.columns)
gmm_inverse3 = pd.DataFrame(scale.inverse_transform(gmmcenter3), columns=scaled.columns)
gmm_inverse3.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%
0,11.541236,7.500422,1.527363,0.697371,1.046388,1.429080,0.523997
1,11.221014,3.935502,1.830610,0.823286,0.321386,1.179470,0.434962
2,16.470462,4.884383,4.966472,1.187913,0.496905,2.354745,0.449660


In [79]:
X_c3.gmm3_cluster.value_counts()

1    290
2    282
0    232
Name: gmm3_cluster, dtype: int64

In [73]:
X_c7 = X.copy()

In [76]:
gmm7 = GaussianMixture(n_components=7, covariance_type='full')
gmm7.fit(scaled)
X_c7['gmm7_cluster'] = gmm7.predict(scaled)
gmmprob7 = pd.DataFrame(gmm7.predict_proba(scaled))
X_c7 = pd.concat([X_c7, gmmprob7], axis=1)

gmmcenter7 = pd.DataFrame(gmm7.means_, columns=scaled.columns)
gmm_inverse7 = pd.DataFrame(scale.inverse_transform(gmmcenter7), columns=scaled.columns)
gmm_inverse7.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%
0,8.700617,5.886663,1.161131,0.617739,0.878376,1.093405,0.513354
1,13.727773,6.062546,1.931683,0.794569,0.485845,1.362134,0.460433
2,18.157534,4.048451,5.171147,1.235469,0.323201,2.420813,0.440929
3,9.523751,2.993247,1.701868,0.725451,0.256239,1.039884,0.426208
4,19.204437,7.805653,5.773714,1.173127,0.958631,2.848312,0.477275
5,15.508432,9.816117,1.755682,0.884936,1.456919,1.959166,0.553128
6,10.940283,3.882072,3.071406,1.100902,0.345275,1.552826,0.436201


In [78]:
X_c7.gmm7_cluster.value_counts()

3    158
2    142
0    135
6    112
1    111
4     84
5     62
Name: gmm7_cluster, dtype: int64

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [80]:
df20st = df[df.GS > 19]
df10min = df20st[df20st.MP > 9.9]
X2 = df10min[['PTS','TRB','AST','STL','BLK','TOV','FG%']]

In [128]:
# answer goes here

X3 = X2.copy(deep=True)

gmm3 = GaussianMixture(n_components=3, covariance_type='full')
gmm3.fit(scaled)
X3['gmm3_cluster'] = gmm3.predict(scaled)
gmmprob3 = pd.DataFrame(gmm3.predict_proba(scaled), index=X2.index)
entropy = entr(gmmprob3)

In [ ]:
gmmprob3

In [130]:
en = entropy.sum(axis=1)

In [ ]:
en.sort_values()

In [132]:
ind = list(en.sort_values().head().index)

In [ ]:
#ind

In [134]:
ind2 = list(en.sort_values().tail().index)

In [ ]:
#ind2

In [136]:
df10min.loc[ind]

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
2042,2016,Rajon Rondo,PG,29,SAC,72,72,35.2,4.9,10.9,0.454,0.9,2.4,0.365,4.1,8.5,0.479,0.494,1.2,2.1,0.580,1.1,5.0,6.0,11.7,2.0,0.1,3.9,2.4,11.9
505,2019,Russell Westbrook,PG,30,OKC,73,73,36.0,8.6,20.2,0.428,1.6,5.6,0.290,7.0,14.5,0.481,0.468,4.1,6.2,0.656,1.5,9.6,11.1,10.7,1.9,0.5,4.5,3.4,22.9
1280,2017,James Harden,PG,27,HOU,81,81,36.4,8.3,18.9,0.440,3.2,9.3,0.347,5.1,9.6,0.530,0.525,9.2,10.9,0.847,1.2,7.0,8.1,11.2,1.5,0.5,5.7,2.7,29.1
2012,2016,Chris Paul,PG,30,LAC,74,74,32.7,7.0,15.1,0.462,1.6,4.4,0.371,5.3,10.6,0.501,0.517,4.0,4.4,0.896,0.5,3.7,4.2,10.0,2.1,0.2,2.6,2.5,19.5
1037,2018,Russell Westbrook,PG,29,OKC,80,80,36.4,9.5,21.1,0.449,1.2,4.1,0.298,8.3,17.0,0.485,0.477,5.2,7.1,0.737,1.9,8.2,10.1,10.3,1.8,0.3,4.8,2.5,25.4


In [137]:
df10min.loc[ind2]

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
515,2019,Marvin Williams,PF,32,CHO,75,75,28.4,3.7,8.7,0.422,1.9,5.1,0.366,1.8,3.6,0.500,0.529,0.9,1.1,0.767,1.0,4.4,5.4,1.2,0.9,0.8,0.6,2.1,10.1
196,2019,Danny Green,SG,31,TOR,80,80,27.7,3.7,7.9,0.465,2.5,5.4,0.455,1.2,2.4,0.487,0.622,0.5,0.6,0.841,0.8,3.2,4.0,1.6,0.9,0.7,0.9,2.1,10.3
181,2019,Rudy Gay,PF,32,SAS,69,51,26.7,5.4,10.8,0.504,1.1,2.7,0.402,4.4,8.1,0.537,0.554,1.7,2.1,0.816,0.9,5.9,6.8,2.6,0.8,0.5,1.7,2.3,13.7
997,2018,Jayson Tatum,SF,19,BOS,80,80,30.5,5.0,10.4,0.475,1.3,3.0,0.434,3.7,7.4,0.492,0.538,2.7,3.2,0.826,0.6,4.4,5.0,1.6,1.0,0.7,1.4,2.1,13.9
752,2018,Rondae Hollis-Jefferson,SF,23,BRK,68,59,28.2,5.0,10.6,0.472,0.2,0.8,0.241,4.8,9.9,0.491,0.481,3.7,4.6,0.788,1.5,5.3,6.8,2.5,1.0,0.7,1.8,2.1,13.9
